# Visión artificial para proteger la identidad de menores

Un problema clásico en visión artificial consiste en la detección de rostros. Recordemos que no es lo mismo detectar que en una imagen hay uno, dos, tres o 20 rostros, que identificar a un individuo concreto: esto sería reconocimiento facial.

El problema que se propone resolver en esta práctica es contar el número de personas (rostros) que hay en una imagen, y para cada una de ellas estimar la edad (busca un modelo pre-entrenado que lo haga satisfactoriamente). Si la persona detectada es menor de edad, el rostro debe difuminarse o pixelarse.

Tu programa debe permitir el tratamiento por lotes, es decir, tomará todas las imágenes en una carpeta de "entrada" y devolverá todas procesadas en una carpeta de "salida". El proceso de cada imagen, con detalle, se mostrará en pantalla. Es decir, se mostrará la imagen con los rostros detectados y la edad estimada de cada persona. Estas imágenes también se guardarán en otra carpeta que se denominará "proceso".

Sube a esta entrega el enlace de Google Colab con la solución que hayas encontrado al problema debidamente documentada.

---

He decidido usar `DeepFace` debido que es la única solución que funciona en el colab. Descargamos `DeepFace`.

In [1]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.8 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=8e9485b8f4cc7281a7ac0420e6926677380095526050190894b847780134fa83
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


Importamos las librerías.

In [2]:
import cv2
import os
import numpy as np
from deepface import DeepFace

25-03-31 21:54:01 - Directory /root/.deepface has been created
25-03-31 21:54:01 - Directory /root/.deepface/weights has been created


Tenemos la función `detect_and_blur_faces` que recibirá los tres ficheros. Donde se van cargando las imágenes y el modelo, tras eso el modelo intenta detectar las caras de la imagen.

Una vez se han obtenido las caras, se intenta estimar la edad y se imprime al lado de la cara. El problema con este modelo es que es bastante malo a la hora de detectar bien las edades de los menores, por lo que he decidio optar por censurar todas las caras de la foto y volcarlas en el `output_data` mientras que las edades se quedaran en el `process_data` para que se haga una comparativa que caras con sus supuestas edades y las censuras.

In [5]:
# Función para detectar y difuminar caras
def detect_and_blur_faces(image_path, output_path, process_data):
    try:
        # Cargar la imagen
        image = cv2.imread(image_path)
        image2 = image.copy()
        if image is None:
            print(f"Error: No se pudo cargar la imagen {image_path}")
            return

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Cargar el modelo de detección de caras de OpenCV
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

        # Detectar caras
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(30, 30))

        for (x, y, w, h) in faces:
            face = image[y:y+h, x:x+w].copy()

            try:
                # Estimar edad con DeepFace
                analysis = DeepFace.analyze(face, actions=["age"], enforce_detection=False)
                age = analysis[0]["age"]

                # Dibujar cuadro y edad estimada
                cv2.rectangle(image2, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(image2, f"Edad: {age}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

                # Aplicar desenfoque a la región de la cara
                blurred_face = cv2.GaussianBlur(face, (99, 99), 30)

                # Crear una máscara con una elipse blanca
                mask = image[y:y+h, x:x+w].copy()
                mask[:] = 0
                cv2.ellipse(mask, (w//2, h//2), (w//2, h//2), 0, 0, 360, (255, 255, 255), -1)

                # Combinar la imagen original y la desenfocada usando la máscara
                face = cv2.bitwise_and(face, face, mask=cv2.bitwise_not(cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)))
                blurred_face = cv2.bitwise_and(blurred_face, blurred_face, mask=cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY))

                # Combinar ambas imágenes
                result = cv2.add(face, blurred_face)
                image[y:y+h, x:x+w] = result

            except Exception as e:
                print(f"Error en análisis de edad para una cara en {image_path}: {e}")
                # Continuar con la siguiente cara si hay un error

        # Guardar la imagen resultante
        cv2.imwrite(output_path, image)
        print(f"Imagen procesada guardada en {output_path}")

        # Guardar la imagen con el proceso
        cv2.imwrite(process_path, image2)
        print(f"Imagen con el proceso guardada en {process_path}")

    except Exception as e:
      print(f"Error al procesar la imagen {image_path}: {e}")


El programa inicia con los ficheros indicados anteriormente, si no los encuentra los crea. Después de eso se intenta cargar las imágenes y procesarlas para dejarlas en el `output_data`.

In [6]:
# Directorios de entrada, salida y proceso
input_dir = "/content/input_data" # Cambia esto a la ruta correcta de tu carpeta de entrada
output_dir = "/content/output_data" # Cambia esto a la ruta correcta de tu carpeta de salida
process_dir = "/content/process_data" # Cambia esto a la ruta correcta de tu carpeta de proceso

# Crea los directorios si no existen
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(process_dir, exist_ok=True)

try:
  # Procesar todas las imágenes en el directorio de entrada
  for filename in os.listdir(input_dir):
      if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
          input_path = os.path.join(input_dir, filename)
          output_path = os.path.join(output_dir, filename)
          process_path = os.path.join(process_dir, filename)
          detect_and_blur_faces(input_path, output_path, process_path)

except FileNotFoundError:
  print("No hay imagenes en el fichero de entrada.")
except Exception as e:
  print(f"Error al procesar las imágenes: {e}")

25-03-31 21:54:39 - age_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/age_model_weights.h5
To: /root/.deepface/weights/age_model_weights.h5
100%|██████████| 539M/539M [00:02<00:00, 253MB/s]


Imagen procesada guardada en /content/output_data/input.png
Imagen con el proceso guardada en /content/process_data/input.png
